# Homework 2

## 2.1

Problem 1

In [56]:
from collections import Counter
import numpy

# Define the corpuses
spam_corpus = [["I", "am", "spam", "spam", "I", "am"], ["I", "do", "not", "like", "that", "spamiam"]]
ham_corpus = [["do", "i", "like", "green", "eggs", "and", "ham"], ["i", "do"]]

def make_spam_filter(good_corpus, bad_corpus):
    # Create the occurence hashes for each corpus
    good_map = Counter(numpy.concatenate(good_corpus))
    bad_map = Counter(numpy.concatenate(bad_corpus))

    # Token probability function taken from reading
    def calc_token_probability(token):
        g = 2 * (good_map.get(token) or 0)
        b = bad_map.get(token) or 0
        ngood = len(good_map)
        nbad = len(bad_map)
        if g + b > 1:
            return max(0.01, min(0.99, min(1.0, b / nbad) / (min(1.0, g / ngood) + min(1.0, b/ nbad))))
        else:
            return 0

    # Table of all the token probabilities
    token_probability = dict(map(
        lambda x: (x, calc_token_probability(x)), numpy.concatenate(good_corpus + bad_corpus)
    ))
    def get_token_probability(token):
        return token_probability.get(token) or 0.5
    
    def spam_probability(message):
        # Get 15 most interesting tokens
        # token_interest = dict(map(
        #    lambda x: (x, abs(tokenProbability(x) - 0.5)), message
        # ))
        probs = list(map(get_token_probability, message))
        prod = numpy.prod(probs)
        print(prod)
        return prod / (prod + numpy.prod(list(map(lambda x: 1 - x, probs))))
        
    
    return spam_probability

check_spam = make_spam_filter(ham_corpus, spam_corpus)
check_spam("I like eggs maybe spam spam".split())

0.0014765419565217393


0.9997668420208968

## 2.2

Problem 2

In [106]:
print("Hello Problem 2")